In [35]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [36]:
spark = SparkSession.builder.appName('solution_ex_09_v1').getOrCreate()

In [37]:
inputPath = './data'
outputPath = './output_ex_09_v1'

print(os.path.exists(inputPath))
print(os.path.exists(outputPath))

True
False


In [38]:
dfRDD = spark.read.load(inputPath, header=False, format="csv", inferschema=True)\
                            .withColumnRenamed("_c0", "sensorID")\
                            .withColumnRenamed("_c1", "date")\
                            .withColumnRenamed("_c2", "PM10")

In [39]:
df_grp = dfRDD.filter('PM10 >= 50')
df_grp.show()

+--------+----------+----+
|sensorID|      date|PM10|
+--------+----------+----+
|      s1|2016-01-01|60.2|
|      s3|2016-01-03|72.9|
|      s1|2016-01-03|55.5|
|      s2|2016-01-03|52.5|
+--------+----------+----+



In [40]:
sensors = df_grp.groupBy('sensorID').agg({'*':'count'}).withColumnRenamed('count(1)','tot_sensors')

sensors.show()

+--------+-----------+
|sensorID|tot_sensors|
+--------+-----------+
|      s2|          1|
|      s3|          1|
|      s1|          2|
+--------+-----------+



In [44]:
sensor = sensors.filter('tot_sensors > 1')
sensor.show()

+--------+-----------+
|sensorID|tot_sensors|
+--------+-----------+
|      s1|          2|
+--------+-----------+



In [45]:
sensor.write.csv(outputPath)
